In [25]:
import pandas as pd
import numpy as np
import spacy
import nltk
import re


In [3]:
#read in data
behold_products = pd.read_excel('Behold+product+data+04262021.xlsx',header = 0)
attr_tags = pd.read_csv('usc_additional_tags.csv')

In [59]:
attr_tags

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic
...,...,...,...,...
97415,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,fit,relaxed
97416,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,lengthblazerscoatsandjackets,midlengthathips
97417,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,closureblazerscoatsandjackets,buttonedfront
97418,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,occasion,work


In [4]:
#pivot df
grouped_tags = pd.DataFrame(attr_tags.groupby(['product_id','attribute_name'])
                            .attribute_value.apply(lambda x:" ".join(set(x)))).reset_index()

pivoted_tags = grouped_tags.pivot(index='product_id',columns=['attribute_name'],values='attribute_value').reset_index()
pivoted_tags.fillna('unknown',inplace=True)

In [52]:
pivoted_tags.sample(10)

attribute_name,product_id,additionalcolor,beltbucklematerial,beltbuckleshape,beltclosure,beltmaterial,beltwidth,calfwidth,category,classbelts,...,subcategorysweater,subcategorysweatshirtandhoodie,subcategorytop,sunglassframematerial,sweatshirtandhoodieclosure,toeexposure,toestyle,trend,uppermaterial,wash
2602,01E2P5TWAMDPP7ZZMNA1XC3WCB,reds blues,unknown,unknown,unknown,unknown,unknown,unknown,shoe,unknown,...,unknown,unknown,unknown,unknown,unknown,closedtoe,roundtoe,unknown,leather,unknown
3812,01E6039NWPFN52FCC9DJ2JSYCC,multi,unknown,unknown,unknown,unknown,unknown,unknown,sweater,unknown,...,pullover,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
2018,01E2M2AEACAXH2DCW50PV5VBNX,unknown,unknown,unknown,unknown,unknown,unknown,unknown,top,unknown,...,unknown,unknown,tee,unknown,unknown,unknown,unknown,unknown,unknown,unknown
641,01DTJ8FHC9A0WKQ9Z30G3CS8GZ,unknown,unknown,unknown,unknown,unknown,unknown,unknown,top,unknown,...,unknown,unknown,tee,unknown,unknown,unknown,unknown,unknown,unknown,unknown
483,01DT50S8KBY1P8M4PZD48A4HH9,unknown,unknown,unknown,unknown,unknown,unknown,unknown,bottom,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
2928,01E4EJMW6A0SZ9MR9R7PFMNG2R,blacks,unknown,unknown,unknown,unknown,unknown,unknown,bottom,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
3057,01E4RV29GQSATQ76JW82SHMSPC,yellows darkbrowns,unknown,unknown,unknown,unknown,unknown,unknown,blazerscoatsjackets,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
1662,01E2KYYSEB56MXQWRWT25RHZZ3,unknown,unknown,unknown,unknown,unknown,unknown,unknown,bottom,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
870,01DVV3WE6HYRSQ5ZNMZ7A11223,golds pinks,unknown,unknown,unknown,unknown,unknown,unknown,accessory,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
2585,01E2P5GAKKTC31DBW7RVQ064GB,whites grays blacks yellows,unknown,unknown,unknown,unknown,unknown,unknown,shoe,unknown,...,unknown,unknown,unknown,unknown,unknown,closedtoe,roundtoe,unknown,synthetic,unknown


In [5]:
pivoted_tags[pivoted_tags.category != 'unknown'].category.unique()

array(['bottom', 'blazerscoatsjackets', 'onepiece', 'accessory', 'shoe',
       'sweater', 'top', 'top bottom', 'sweatshirthoodie',
       'top accessory', 'top blazerscoatsjackets',
       'sweater blazerscoatsjackets', 'top sweatshirthoodie'],
      dtype=object)

In [61]:
(pivoted_tags != 'unknown').apply(sum).sort_values(ascending=False)[1:20]

attribute_name
category             3969
primarycolor         3925
style                3916
occasion             3914
gender               3913
fit                  2949
sizing               2941
materialclothing     2924
drycleanonly         2776
sleevelength         2047
neckline             1774
additionalcolor      1532
lengthtop            1354
pattern              1346
subcategorytop        952
subcategorybottom     892
subcategoryshoe       669
shoewidth             666
toestyle              661
dtype: int64

In [27]:
#function to tokenize columns, removing stopwords and returning lemmas
nlp = spacy.load("en_core_web_md", disable=['ner', 'parser', "tok2vec"])
def spacy_tokens(line):
    line = re.findall('(?u)\\b[a-zA-Z][a-zA-Z]+\\b',str(line))
    doc = nlp(" ".join(line))
    return " ".join([token.lemma_.lower() for token in doc if token.is_stop != True])
    
spacy_tokens('Our signature khadi shirt\navailable in black .')

'signature khadi shirt available black'

In [28]:
#%%time 1m08s per column
behold_products['token_description'] = behold_products['description'].apply(spacy_tokens)

In [32]:
#build token columns
df_cols = ['description','brand_name','name']

#runs a little slow
def token_builder(df,df_cols):
    for i in df_cols:
        print('parsing token_'+i)
        df['token_'+i] = df[i].apply(spacy_tokens)
        


In [33]:
%%time
token_builder(behold_products,df_cols)

parsing token_description
parsing token_brand_name
parsing token_name
Wall time: 1min 41s


In [58]:
#136 instances where name not identical to brandname
behold_products[['name','brand_name']][behold_products['name'] != behold_products['brand_name']]

,name,brand_name
18072,OLD Behold Demi Lune Miel,Behold Demi Lune Miel
18073,OLD Behold Flat Hobo Miel,Behold Flat Hobo Miel
18074,OLD Behold Flat Hobo Scarab,Behold Flat Hobo Scarab
18075,OLD Behold Lune Shopper Black Grain,Behold Lune Shopper Black Grain
18076,OLD Behold Lune Shopper Plantation Grain,Behold Lune Shopper Plantation Grain
...,...,...
54483,Ilana Mary Jane,Chocolate Croco
55180,Travel Ballerina,Off White Woven Sheepskin
55181,Travel Ballerina,Off White Sheepskin
55340,CARMEN SANDAL,Black Sheepskin


In [34]:
behold_products[['product_id','brand','token_description','token_brand_name','token_name']]

,product_id,brand,token_description,token_brand_name,token_name
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,signature khadi shirt available black white ea...,khadi stripe shirt signature shirt,khadi stripe shirt signature shirt
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,mid length dress ruffles adjustable straps bia...,ruffle market dress loopy pink sistine tomato,ruffle market dress loopy pink sistine tomato
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,ibi slip black knit sneaker women,ibi slip black knit sneaker women,ibi slip black knit sneaker women
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,nan,catiba pro skate black suede canvas contrast t...,catiba pro skate black suede canvas contrast t...
...,...,...,...,...,...
61350,01EYB5ERGYPFNGM6C9QK7Q9EV0,Nomasei,flat bowvida mule black suede ideal spring sum...,bowvida mule black suede kidskin,bowvida mule black suede kidskin
61351,01EHWTBFP368Q035FW95TRJDAY,Nomasei,flat vida mule tangerine suede comfortable fem...,sandale vida mule tangerine suede kidskin,sandale vida mule tangerine suede kidskin
61352,01EHWTCFTPPSCW10D4XBQZF28H,Nomasei,flat bowvida mule fuschia suede ideal spring s...,bowvida mule fuschia suede kidskin,bowvida mule fuschia suede kidskin
61353,01EYB5B5FH7JESXF82ZEMVXZMS,Nomasei,flat vida mule silver metalized leather comfor...,vida mule silver metalized leather,vida mule silver metalized leather


In [35]:
#identify colors and products
#from hw 3

colors = ['Beige','Black','Blue','Brown','Burgundy','Gold','Gray','Green','Multi','Navy','Neutral','Orange','Pink','Purple','Red','Silver','Teal','White','Yellow','leopard','striped','plaid','floral']

products =  ['Bottom','One Piece','Shoe','Handbag','Scarf','skirt','hoodie','jacket','coat','pajamas', 'blazer',
            'dress','top','blouse','bag','purse','earring','lingerie','bra','jewelry','pants','jewelry','accessories',
           'jeans','belt','raincoat','outerwear','Two Piece', 'hat','sneaker','heels','pumps','camisole','outerwear','sweater']


def stem_join(list):
    doc = nlp(" ".join(list))
    list = [token.lemma_.lower() for token in doc]
    list = '|'.join(r"\b{}\b".format(x) for x in list) #from stackoverflow
    return list

colors = stem_join(colors)
products = stem_join(products)

In [37]:
#identify products and colors
behold_products['product type'] = behold_products['token_name'].str.findall(products)
#data['colors'] = data['token_descriptions'].apply(lambda x:" ".join(x)).str.findall(colors)

0            []
1       [dress]
2     [sneaker]
3     [sneaker]
4     [sneaker]
5            []
6            []
7            []
8            []
9            []
10           []
11           []
12           []
13           []
14           []
15      [skirt]
16      [dress]
17      [dress]
18      [pants]
19           []
20      [dress]
21      [dress]
22      [skirt]
23           []
24      [dress]
25      [dress]
26      [pants]
27           []
28      [dress]
29      [skirt]
30           []
31      [dress]
32    [sweater]
33    [sweater]
34    [sweater]
35           []
36      [dress]
37           []
38      [dress]
39      [dress]
40      [dress]
41           []
42      [dress]
43      [dress]
44      [dress]
45      [dress]
46      [dress]
47      [dress]
48      [dress]
49      [dress]
Name: token_name, dtype: object

In [ ]:
behold_products['token_description'][:100].str.findall(products)

In [ ]:
behold_products.iloc[95,:]